<a href="https://colab.research.google.com/github/sherry975/CSC-791-Project/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# !ls "/content/drive/My Drive/"

import os
os.chdir("/content/drive/My Drive/CSC791-NLP/791 NLP project/791_project_repo")


# ------------------------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
import sys
import traceback

import re
import datetime
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('punkt')

from sklearn import preprocessing
from numpy import dot
from numpy.linalg import norm



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# process raw function words list and generate a txt

def get_function_words_list(filename):
    func_words_lst = []
    with open(filename, 'r', encoding='utf-8', newline='') as f:
        content = f.readlines()
        for row in content:
            if row[0] != "#":
                word = row.split(" ")[0]
                func_words_lst.append(word)
    return func_words_lst            


# test
func_words_lst = get_function_words_list("raw_data/james_o_shea_277.txt")
func_words_set = set(func_words_lst)

print(len(func_words_lst), func_words_lst)
print(len(func_words_set))


277 ['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'be', 'became', 'because', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'but', 'by', 'can', 'cannot', 'could', 'dare', 'despite', 'did', 'do', 'does', 'done', 'down', 'during', 'each', 'eg', 'either', 'else', 'elsewhere', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'first', 'for', 'former', 'formerly', 'from', 'further', 'furthermore', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereabouts', 'hereafter', 'hereby', 'herein', 'hereinafter', 'heretofore', 'hereunder', 'hereupon', 'herewith', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'i', '

In [ ]:
# split data into train/test
import re
import statistics
import random

FORMALITY_MEDIAN = 4.2

def clean_text(orig): 
    cleaned = orig
    
    if re.match('^[0-9].\)', orig[:3]):
        cleaned = orig[3:]
    elif re.match('^[0-9][0-9][A-Z]' ,orig[:3]) or re.match('^[0-9].' ,orig[:2]) or re.match('^[0-9]\)', orig[:2]):
        cleaned = orig[2:]
    elif re.match('^[0-9][A-Z]', orig[:2]):
        cleaned = orig[1:]
    
    return cleaned

def split_train_test(filename):
    low_formality = []
    high_formality = []
    train_set = []
    test_set = []
        
    reader = csv.reader(open(filename, newline='', encoding='mac_roman'), delimiter=',',
                        quotechar='"')
    row_cnt = 0
    
    try:
        for row in reader:
            row_cnt += 1
            
            if "sentence" in row[0].lower():
                continue
                
#             if row_cnt > 10:
#                 break
                
            # col: 0 Sentence ID	1 HIT ID	2 Formality	Informativeness	Implicature	5 Length in Words	6 Length in Characters	F-score	I-score	Lexical Density	11 Sentence
            sentence = row[10]
            formality = float(row[2])
            cleaned_text = clean_text(sentence)
            
            if formality <= FORMALITY_MEDIAN:
                # setence_id, sentence_text, len_words, len_char, formality, label
                low_formality.append([row[0], cleaned_text, row[5], row[6], formality, "0"])
            else:
                high_formality.append([row[0], cleaned_text, row[5], row[6], formality, "1"])
            

        print("# low formality, # hgih formality: ", len(low_formality),len(high_formality))
        
        random.shuffle(low_formality)
        random.shuffle(high_formality)
        
        num_train_low = int(len(low_formality) * 0.9)
        num_train_high = int(len(high_formality) * 0.9)
        train_set = low_formality[:num_train_low] + high_formality[:num_train_high]
        test_set = low_formality[num_train_low:] + high_formality[num_train_high:]
        
        random.shuffle(train_set)
        random.shuffle(test_set)
        
        return train_set, test_set 

    except Exception as ex:
        sys.stderr.write('Exception\n')
        extype, exvalue, extrace = sys.exc_info()
        traceback.print_exception(extype, exvalue, extrace)
    
train_set, test_set = split_train_test("raw_data/mturk_experiment_2.csv")

print("# train, # test: ", len(train_set), len(test_set))

headers = ["sentence_id", "text", "len_in_words", "len_in_char", "formality", "label"]

with open('input_data/training.csv', 'w') as csvoutput:
    writer = csv.writer(csvoutput)
    writer.writerow(headers)
    for item in train_set:
        writer.writerow(item)
    
with open('input_data/testing.csv', 'w') as csvoutput:
    writer = csv.writer(csvoutput)
    writer.writerow(headers)
    for item in test_set:
        writer.writerow(item)

3830 3202
6328 704
